Ссылка: https://www.youtube.com/watch?v=sIbzKA6MId8

# SQLAlchemy

`SQLAlchemy` - это библиотке `Python` для работы с БД, который использует `ORM`, который позволяет работать с базой при помощи языка `Python`, а не `SQL`.

## Определение модели

<b>Недекларативный способ</b> - сначала следует создание объекта таблицы, потом связываем ее с БД, а после связываем с классами, созданными нами.  
<b>Декларативный способ</b> - связка модели и таблицы происходит непосредственно в классе одновременно.  
Чаже всего используютДекларативный подход.

Вот пример создания БД:

In [1]:
import sqlalchemy as db 


metadata = db.MetaData() # информация о БД и ее объектах

authors = db.Table("authors", metadata,
    db.Column("id_author", db.Integer, primary_key=True),
    db.Column("name", db.String(250))
)

books = db.Table("books", metadata,
    db.Column("id_book", db.Integer, primary_key=True),
    db.Column("title", db.String(250), nullable=False),
    db.Column("author_id", db.Integer, db.ForeignKey("authors.id_author")),
    db.Column("genre", db.String(250)),
    db.Column("price", db.Integer)
)

engine = db.create_engine('sqlite:///books.db')
metadata.create_all(engine)

conneection = engine.connect()

insert_author_query = authors.insert().values([
    {"name": "Lutz"}
])
conneection.execute(insert_author_query)

insert_books_query = books.insert().values([
    {"title": "Learn Python", "author_id": 1, "genre": "Education", "price": 1299},
    {"title": "Clear Python", "author_id": 1, "genre": "Education", "price": 956},
])
conneection.execute(insert_books_query)



c:\temp1\ipykernel_12588\33293353.py:27: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conneection.execute(insert_author_query)


Получим данные:

In [2]:
books_gr_1000_query = books.select().where(books.columns.price > 1000)
result = conneection.execute(books_gr_1000_query)
print(result.fetchall())

[(1, 'Learn Python', 1, 'Education', 1299), (3, 'Learn Python', 1, 'Education', 1299), (5, 'Learn Python', 1, 'Education', 1299), (7, 'Learn Python', 1, 'Education', 1299)]


Еще получим данные:

In [3]:
join_select = db.select([authors, books]).where(authors.columns.id_author == books.columns.author_id)
result_join = conneection.execute(join_select)
print(result_join.fetchall())

[(1, 'Lutz', 1, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 2, 'Clear Python', 1, 'Education', 956), (1, 'Lutz', 3, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 4, 'Clear Python', 1, 'Education', 956), (1, 'Lutz', 5, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 6, 'Clear Python', 1, 'Education', 956), (1, 'Lutz', 7, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 8, 'Clear Python', 1, 'Education', 956)]


## Подключение и работа с существующей БД

Вот пример:

In [6]:
import sqlalchemy as db


engine = db.create_engine("sqlite:///books.db")
metadata = db.MetaData(engine) # помещаем данные БД в объект MetaData

# загружаем таблицы в переменные
authors = db.Table("authors", metadata, autoload=True)
books = db.Table("books", metadata, autoload=True)

# создаем подключение
connection = engine.connect()

# создание объекта выборки, обратите внимание на db.and_
select_query = db.select([authors, books]).where(db.and_(authors.columns.id_author == books.columns.author_id, books.columns.price > 1000))
result = connection.execute(select_query)
print(result.fetchall())

[(1, 'Lutz', 1, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 3, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 5, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 7, 'Learn Python', 1, 'Education', 1299)]


Удаление данных:

In [8]:
delete_query = books.delete().where(books.columns.author_id == 1)
connection.execute(delete_query)

Обновление данных:

In [9]:
update_query = books.update().where(books.columns.id_book == 1).values("AnotherTitle")
connection.execute(update_query)

## Связывание с классами

В данных пример разберем случае, когда все взаимодействие с БД будет происходить с помощью сессий.
Вот пример:

In [ ]:
from sqlalchemy.orm import mapper, relationship, sessionmaker
import sqlalchemy as db


engine = db.create_engine("sqlite:///books.db")
metadata = db.MetaData(engine)

# загружаем таблицы в переменные
authors = db.Table("authors", metadata, autoload=True)
books = db.Table("books", metadata, autoload=True)

class Book():
    def __init__(self, title, author_id, genre, price) -> None:
        self.title = title
        self.author_id = author_id
        self.genre = genre
        self.price = price

    def __repr__(self) -> str:
        return "<Book ('%s', '%s', '%s' , '%s')>" % (self.title, str(self.author_id), self.genre, str(self.price))
    

class Authors():
    def __init__(self, name) -> None:
        self.name = name

    def __repr__(self) -> str:
        return "<Author '%s'" % self.name
    

# связываем классы и таблицы
mapper(Book, books)
mapper(Authors, authors)

# добавим книгу в БД объектным способом
new_book = Book("NewBook", 1, "NewG", 2500)

# используем сессии для связи с БД
DBSession = sessionmaker(bind=engine)

# создаем объект созданной сессии
session = DBSession()

# добавляем запись в таблицу
session.add(new_book)

# комитим изменения
session.commit()

Сделаем `select`:

In [16]:
for row in session.query(Book).filter(Book.price > 1000):
    print(row)

<Book ('NewBook', '1', 'NewG' , '2500')>
<Book ('NewBook', '1', 'NewG' , '2500')>
<Book ('NewBook', '1', 'NewG' , '2500')>
<Book ('NewBook', '1', 'NewG' , '2500')>
<Book ('NewBook', '1', 'NewG' , '2500')>
<Book ('NewBook', '1', 'NewG' , '2500')>


Сложный запрос с несколькитми улсовиями:

In [4]:
from sqlalchemy.orm import mapper, relationship, sessionmaker
import sqlalchemy as db


engine = db.create_engine("sqlite:///books.db")
metadata = db.MetaData(engine)

authors = db.Table("authors", metadata, autoload=True)
books = db.Table("books", metadata, autoload=True)

class Book():
    def __init__(self, title, author_id, genre, price) -> None:
        self.title = title
        self.author_id = author_id
        self.genre = genre
        self.price = price

    def __repr__(self) -> str:
        return "<Book ('%s', '%s', '%s' , '%s')>" % (self.title, str(self.author_id), self.genre, str(self.price))
    

class Authors():
    def __init__(self, name) -> None:
        self.name = name

    def __repr__(self) -> str:
        return "<Author '%s'" % self.name
    

mapper(Book, books)
mapper(Authors, authors)

new_book = Book("NewBook", 1, "NewG", 2500)

DBSession = sessionmaker(bind=engine)

session = DBSession()

session.add(new_book)

session.commit()

# запрос
for row in session.query(Book, Authors).filter(Book.author_id == Authors.id_author).filter(Book.price > 1000):
    print(row.Book.title, row.Authors.name)

NewBook Lutz
NewBook Lutz
NewBook Lutz
NewBook Lutz
NewBook Lutz
NewBook Lutz
NewBook Lutz


c:\temp1\ipykernel_19252\1079631501.py:6: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  metadata = db.MetaData(engine)


Запрос на обновление данных:

In [5]:
query_book = session.query(Book).filter(Book.id_book == 3).one()
if query_book != []:
    query_book.price = 3000
    session.add(query_book)
    session.commit()

Запрос на удаление данных:

In [6]:
query_book = session.query(Book).filter(Book.id_book == 3).one()
if query_book:
    print(query_book)
    session.delete(query_book)
    session.commit()

<Book ('NewBook', '1', 'NewG' , '3000')>


## Декларативное определение

Данный подход короче и им пользуются чаще всего.  
Связывание таблицы в базе с пользовательским классом происходит в моменте его создания.

Вот пример:

In [8]:
import sqlalchemy as db
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship


# новый объект БД
engine = db.create_engine("sqlite:///books2.db", echo=True)
Base = declarative_base()


class Book(Base):
    ''' присваиваем имя таблицы в БД и одновременно связываем ее с классом '''
    __tablename__ = "Books"

    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(250), nullable=False)
    author_id = db.Column(db.Integer, db.ForeignKey("Authors.id"))
    genre = db.Column(db.String(250))
    priec = db.Column(db.Integer, nullable=False)
    author = relationship("Author")


class Author(Base):
    ''' присваиваем имя таблицы в БД и одновременно связываем ее с классом '''
    __tablename__ = "Authors"

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(250), nullable=False)
    book = relationship("Book") # связь 1 ко многим


# создаем БД и записываем изменения в БД
Base.metadata.create_all(engine)

2023-08-12 11:21:42,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-12 11:21:42,231 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Books")
2023-08-12 11:21:42,231 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-12 11:21:42,239 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Books")
2023-08-12 11:21:42,239 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-12 11:21:42,239 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Authors")
2023-08-12 11:21:42,239 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-12 11:21:42,239 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Authors")
2023-08-12 11:21:42,239 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-12 11:21:42,239 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Authors" (
	id INTEGER NOT NULL, 
	name VARCHAR(250) NOT NULL, 
	PRIMARY KEY (id)
)


2023-08-12 11:21:42,247 INFO sqlalchemy.engine.Engine [no key 0.00103s] ()
2023-08-12 11:21:42,255 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Books" (
	id I